In [69]:
from urllib.parse import quote, urlparse
import urllib.request
import oos, requests
import os, sys

import datetime

In [ ]:
def download_file(url, filename, timeout=3600):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
        return True
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))

        try:
            response = requests.Session().get(url, timeout=timeout)
            if(response.status_code!=200):
                print('Error response, code = %d, body = %s' % (response.status_code, response.text))
                return False
        except RequestException as e:
            sys.stdout.write("Couldn't read %s because %s" % (url, e))
            return False

        open(filename + '.tmp', "wb").write(response.content)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(response.content), filename))
        return True

            
class RootArg:
    arg_dict = {
        'v': None,
        't': 0,
        'ps': 0,
        'l': None,
        'bt': None,
        'et': None,
        'startDwell': 0,
        'endDwell': 0,
        'fps': 30
    }
    def __init__(self, v, l, bt, et, **kwargs):
        self.arg_dict['v'] = v
        self.arg_dict['l'] = l
        self.arg_dict['bt'] = bt
        self.arg_dict['et'] = et
        
    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://headless.earthtime.org/#' + "&".join(arg_list)
    
class ThumbnailUrl:
    arg_dict = {
        'root': None,
        'width': None,
        'height': None,
        'format': 'png',
        'fps': 30,
        'tileFormat': 'mp4',
        'startDwell': 0,
        'endDwell': 0,
        'fromScreenshot': True
    }
    
    def __init__(self, root, width, height):
        self.arg_dict['root'] = root
        self.arg_dict['width'] = width
        self.arg_dict['height'] = height
        

    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://hal40-thumbnailworker.createlab.org/thumbnail?' + "&".join(arg_list)
    

def export_landsat_images_from_earthtime(sharelink, odir=None, w='5120', h='2880'):
    years = range(1984,2023)
    dates = ['%s0401' % x for x in years]
    u = urlparse(url)
    v = None
    l = None
    for fragment in u.fragment.split('&'):
        key,val = fragment.split('=')
        if key == 'v':
            v = val
        if key == 'l':
            l = val
    if v is None or l is None:
        print('Malformed sharelink')
        return False
    
    if odir is None:
        now = datetime.datetime.now()
        timestamp = int(now.timestamp())
        odir = '%s_%s' % (l.replace(',','_'), timestamp)

    os.makedirs(odir, exist_ok=True)
    thumbnails = []
    for date in dates:
        bt = et = date
        root_arg = RootArg(v,l,bt,et)
        thumbnail_url = ThumbnailUrl(quote(root_arg.get(), safe=''), w, h)
        fname = str(date[0:4]) + '.png'
        thumbnails.append((thumbnail_url.get(), os.path.join(odir,fname)))
    for t in thumbnails:
        download_file(t[0], t[1])
    return True
    
    
    

In [72]:
export_landsat_images_from_earthtime('https://earthtime.org/explore#v=36.15028,-114.54353,9.608,latLng&t=3.2&ps=50&l=blsat&bt=19840101&et=20221231')

Done, wrote 36542429 bytes to blsat_1705948888.84853/1984.png
Done, wrote 36015642 bytes to blsat_1705948888.84853/1985.png
Done, wrote 37399150 bytes to blsat_1705948888.84853/1986.png
Done, wrote 36466677 bytes to blsat_1705948888.84853/1987.png
Done, wrote 36483180 bytes to blsat_1705948888.84853/1988.png
Done, wrote 36681954 bytes to blsat_1705948888.84853/1989.png
Done, wrote 37157731 bytes to blsat_1705948888.84853/1990.png
Done, wrote 37163973 bytes to blsat_1705948888.84853/1991.png
Done, wrote 37260431 bytes to blsat_1705948888.84853/1992.png
Done, wrote 36791456 bytes to blsat_1705948888.84853/1993.png
Done, wrote 36846887 bytes to blsat_1705948888.84853/1994.png
Done, wrote 37138750 bytes to blsat_1705948888.84853/1995.png
Done, wrote 36844498 bytes to blsat_1705948888.84853/1996.png
Done, wrote 37219141 bytes to blsat_1705948888.84853/1997.png
Done, wrote 37316115 bytes to blsat_1705948888.84853/1998.png
Done, wrote 38141162 bytes to blsat_1705948888.84853/1999.png
Done, wr

Done, wrote 39402009 bytes to blsat_1705948888.84853/2002.png
Done, wrote 39601119 bytes to blsat_1705948888.84853/2003.png
Done, wrote 39512307 bytes to blsat_1705948888.84853/2004.png
Done, wrote 38398532 bytes to blsat_1705948888.84853/2005.png
Done, wrote 39341526 bytes to blsat_1705948888.84853/2006.png
Done, wrote 39831096 bytes to blsat_1705948888.84853/2007.png
Done, wrote 39204882 bytes to blsat_1705948888.84853/2008.png
Done, wrote 39441735 bytes to blsat_1705948888.84853/2009.png
Done, wrote 39176688 bytes to blsat_1705948888.84853/2010.png
Done, wrote 39368818 bytes to blsat_1705948888.84853/2011.png
Done, wrote 41290764 bytes to blsat_1705948888.84853/2012.png
Done, wrote 40888712 bytes to blsat_1705948888.84853/2013.png
Done, wrote 40962617 bytes to blsat_1705948888.84853/2014.png
Done, wrote 41795592 bytes to blsat_1705948888.84853/2015.png
Done, wrote 40673678 bytes to blsat_1705948888.84853/2016.png
Done, wrote 41784717 bytes to blsat_1705948888.84853/2017.png
Done, wr

Done, wrote 40160111 bytes to blsat_1705948888.84853/2020.png
Done, wrote 41428189 bytes to blsat_1705948888.84853/2021.png
Done, wrote 42347894 bytes to blsat_1705948888.84853/2022.png


True